In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup 
import re
import time
import pandas as pd
from datetime import datetime
import openpyxl
from selenium.common.exceptions import ElementClickInterceptedException

def tranJsPathToText(jsPath):
   element = driver.execute_script(jsPath)
   if element:
       result =  element.text
   else:
       result = ''
   return result

def tranJsPathToSrc(jsPath):
   element = driver.execute_script(jsPath)
   if element:
       result =  element.get_attribute('src') 
   else:
       result = ''
   return result

def trim_string(text):
    """
    문자열에서 'GM' 왼쪽의 모든 문자와 '_img' 오른쪽의 모든 문자를 제거하는 함수입니다.
    
    :param text: 입력 문자열
    :return: 'GM'으로 시작하고 '_img'로 끝나는 부분만 남긴 문자열
    """
    # 'GM'으로 시작하는 인덱스 찾기
    start_index = text.find('GM')
    
    if start_index == -1:
        return ''  # 'GM'이 없는 경우 빈 문자열 반환
    
    # '_img'로 끝나는 인덱스 찾기
    end_index = text.find('_img', start_index)
    
    if end_index == -1:
        return ''  # '_img'가 없는 경우 빈 문자열 반환
    
    # '_img'까지 포함하여 문자열 자르기
    end_index += len('_img')  # '_img'의 끝 인덱스까지 포함
    
    # 결과 문자열 반환
    return text[start_index:end_index]

try: 
    driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

    url = "https://living.hyundailivart.co.kr/index"
    driver.get(url)
    time.sleep(2)
    ul = driver.find_element(By.XPATH, '//*[@id="lstore-category-listsD100000584"]/div[3]/ul') 
    li = ul.find_elements(By.XPATH, './li')
    data = []
    current_date = datetime.now().strftime('%Y-%m-%d')

    # for i in range(0,2):
    for i in range(len(li)):
        var_i = str(i+1)
        # print(var_i)
        # s = '//*[@id="lstore-category-listsD100000584"]/div[3]/ul/li[' + var_i + ']'
        s = '//*[@id="lstore-category-listsD100000584"]/div[3]/ul/li['+var_i+']/a'
        li_click = driver.find_element(By.XPATH, s)
        time.sleep(2)

        attempts = 0
        while attempts < 30:  # 최대 3번 시도
            try:
                li_click.click()
                break  # 성공하면 반복 종료
            except ElementClickInterceptedException:
                time.sleep(1)  # 잠시 대기 후 재시도
                attempts += 1
        

        time.sleep(2)
        js = 'return document.querySelector("#container > div.container > div > div > div.detail-search.p-data > section > ul")'
        jsResult =  driver.execute_script(js)
        time.sleep(2)
        if jsResult is not None:
            jsResultHtml = BeautifulSoup(jsResult.get_attribute('outerHTML'), 'html.parser')
            time.sleep(2)
        else:
            js ='return document.querySelector("#container > div.container > div > div > div > section > ul")'
            jsResult =  driver.execute_script(js)
            jsResultHtml = BeautifulSoup(jsResult.get_attribute('outerHTML'), 'html.parser')
            time.sleep(2)

        # li 태그를 모두 찾아 출력
        li_tags = jsResultHtml.find_all('li')
        global exceptVar
        for i in range(len(li_tags)):
        # for i in range(1):
            exceptVar = ''
            row = []
            # 각 <li> 요소 클릭
            var_i = str(i+1)
            # s = '//*[@id="container"]/div[2]/div/div/div/section/ul/li['+var_i+']'
            s = 'return document.querySelector("#container > div.container > div > div > div.detail-search.p-data > section > ul > li:nth-child('+var_i+') > div > div.product-item-image")'
            print(var_i + "번쨰 상품.....................................................................................................")
            li_to_click  = driver.execute_script(s)
            time.sleep(1)
            li_to_click.click()

            #상품상세 데이터 가져오기
        
            try : 
                # MDL_CD
                jsPath = 'return document.querySelector("#container > div.contents-items-wrap > div:nth-child(1) > section > div.fr > div.pitem-header-info > div.pitem-header-section.pitem-header-section--code > dl > dd > span")'
                tempText = tranJsPathToText(jsPath)
                # print(tempText)
                row.append(tempText)

                # MDL_NM
                jsPath = 'return document.querySelector("#container > div.contents-items-wrap > div:nth-child(1) > section > div.fr > div.pitem-header-info > div:nth-child(2) > div > div.is-flex > h2")'
                tempText = tranJsPathToText(jsPath)
                # print(tempText)
                row.append(tempText)

                # CTG_ID
                jsPath = 'return document.querySelector("#container > div.location-header > div > nav > ol > li:nth-child(2) > div > div.dropdown-trigger > button > span:nth-child(1)")'
                tempText = tranJsPathToText(jsPath)
                # print(tempText)
                row.append(tempText)

                # IMG_PATH
                jsPath = 'return document.querySelector("#container > div.contents-items-wrap > div:nth-child(1) > section > div.fl > div.pitem-header-photo > div.gallery-photo.xzoom-container.swiper-container-initialized.swiper-container-horizontal > ul > li.swiper-slide.swiper-slide-active > img")'
                src = tranJsPathToSrc(jsPath)
                cleaned_text = src.split('.jpg')[0] + '.jpg'
                # print(cleaned_text)
                row.append(cleaned_text)

                # IMG_NM
                jsPath = 'return document.querySelector("#container > div.contents-items-wrap > div:nth-child(1) > section > div.fl > div.pitem-header-photo > div.gallery-photo.xzoom-container.swiper-container-initialized.swiper-container-horizontal > ul > li.swiper-slide.swiper-slide-active > img")'
                src = tranJsPathToSrc(jsPath)
                imgNm = trim_string(src)
                if imgNm == '':
                    imgNm = 'imgNm'
                # print(imgNm)
                row.append(imgNm)

                # MDL_COLOR
                jsPath = 'return document.querySelector("#content-gosi-detail-1 > div > div:nth-child(2) > dl:nth-child(2) > dd")'
                tempText = tranJsPathToText(jsPath)
                # print(tempText)
                row.append(tempText)
                #MDL_DETAIL
                row.append('MDL_DETAIL')
                #VLID_STRT_DT
                row.append(current_date)
                # row.append('')
                #VLID_END_DT
                row.append(current_date)
                # row.append('')
                # COST
                jsPath = 'return  document.querySelector("#container > div.contents-items-wrap > div:nth-child(1) > section > div.fr > div.pitem-header-info > div:nth-child(3) > div.pitem-header-section__price--wrap.space-between > div > span.pitem-header-section__price")'
                tempText = tranJsPathToText(jsPath)
                cost = int(tempText.splitlines()[0].replace(',',''))
                if cost == '':
                    cost = 100000
                # print(cost)
                row.append(cost)

                #DEL_YN
                row.append('N')
                #FRST_REG_USER_ID
                row.append('SYSTEM')
                #FRST_REG_DATE
                row.append(current_date)
                # row.append('')
                #LAST_MOD_USER_ID
                row.append('SYSTEM')
                #LAST_MOD_DATE
                row.append(current_date)
                # row.append('')

            except  Exception as e:
                print("예외발생")
                if isinstance(exceptVar, str):  # 먼저 문자열인지 확인
                    if exceptVar.isdigit():  # 숫자인지 확인
                        exceptVar = 100000
                    elif exceptVar.isalpha():  # 글자인지 확인
                        exceptVar = "tempStr"
                    else:
                        exceptVar = ''
                else:
                    exceptVar = ''
                row.append(exceptVar)
            finally:
                for r in row:
                    print(r)
                data.append(row)
            # 뒤로 가기 버튼 클릭 (JavaScript 방식으로 대체)
            driver.execute_script("window.history.go(-1);")
        # 뒤로 가기 버튼 클릭 (JavaScript 방식으로 대체)
        driver.execute_script("window.history.go(-1);")    
    
except  Exception as e:
    df = pd.DataFrame(data, columns=['MDL_CD','MDL_NM','CTG_ID','IMG_PATH','IMG_NM','MDL_COLOR','MDL_DETAIL','VLID_STRT_DT','VLID_END_DT','COST','DEL_YN','FRST_REG_USER_ID','FRST_REG_DATE','LAST_MOD_USER_ID','LAST_MOD_DATE'])
    df.to_excel('product.xlsx', index=False)
    df.to_csv('product.csv',
            sep=',',            # 필드 구분자
            encoding='utf-8',   # 파일 인코딩
            index=False,        # 인덱스 저장 여부
            header=True        # 헤더 저장 여부
        
    )
    

driver.quit()


1번쨰 상품.....................................................................................................
P200092739
[아울렛] 한정수량★로데 3인 패브릭 소파 (2색)
소파
https://static.hyundailivart.co.kr/upload_mall/goods/P200092739/GM41924557_img.jpg
GM41924557_img
오션리지그레이샌드
MDL_DETAIL
2024-09-04
2024-09-04
439600
N
SYSTEM
2024-09-04
SYSTEM
2024-09-04
2번쨰 상품.....................................................................................................
P200165393
베이(Bae) 1인 패브릭 소파
소파
https://static.hyundailivart.co.kr/upload_mall/goods/P200165393/GM43256779_img.jpg
GM43256779_img

MDL_DETAIL
2024-09-04
2024-09-04
1440000
N
SYSTEM
2024-09-04
SYSTEM
2024-09-04
3번쨰 상품.....................................................................................................
P200165382
쿠스(Kuss) 3.5인 패브릭 소파
소파
https://static.hyundailivart.co.kr/upload_mall/goods/P200165382/GM43233169_img.jpg
GM43233169_img

MDL_DETAIL
2024-09-04
2024-09-04
3670000
N
SYSTEM
2024-09-04
SYSTEM
2024-09-04
4번쨰 상품..................